# Auto Generated Agent Chat: Society of Minds

This notebook showcases how to use conversable agents in AutoGen to realize a Society of Mind (SoM) scenario, in which an agent runs a  chat session with another agent as an inner-monologue. The idea of SoM and part of the example demonstrated in this notebook come from this PR: https://github.com/microsoft/autogen/pull/890
 

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

## Society of Minds

In [1]:
import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
llm_config = {"config_list": config_list, "cache_seed": 44}

financial_tasks = [
    """On which days this year was Microsoft Stock higher than $400? Put results in a table.""",
    """Investigate the possible reasons of the stock performance.""",
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

### Construct a group chat manager to be used for the inner-monologue

In [2]:
assistant = autogen.AssistantAgent(
    "Inner-assistant",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

code_interpreter = autogen.UserProxyAgent(
    "Inner-code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

groupchat = autogen.GroupChat(
    agents=[assistant, code_interpreter],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
)

### Example 1: Solve one task with SoM

In [3]:
financial_assistant_1 = autogen.AssistantAgent(
    name="Financial_assistant_1",
    llm_config={"config_list": config_list},
    # is_termination_msg=lambda x: x.get("content", "") == "",
)

financial_assistant_2 = autogen.AssistantAgent(
    name="Financial_assistant_2",
    llm_config={"config_list": config_list},
    # is_termination_msg=lambda x: x.get("content", "") == "",
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

financial_assistant_1.register_nested_chats(
    [autogen.Agent, None], [{"recipient": manager, "summary_method": "reflection_with_llm"}]
)
user.initiate_chat(financial_assistant_1, message=financial_tasks[0])

User (to Financial_assistant_1):

On which days this year was Microsoft Stock higher than $400? Put results in a table.

--------------------------------------------------------------------------------

********************************************************************************
Start a new chat with the following message: 
On which days this year was Microsoft Stock higher than $400? Put results in a table.

With the following carryover: 


********************************************************************************
Financial_assistant_1 (to chat_manager):

On which days this year was Microsoft Stock higher than $400? Put results in a table.

--------------------------------------------------------------------------------
Inner-assistant (to chat_manager):

To determine on which days Microsoft (ticker symbol: MSFT) stock was trading higher than $400 this year, I will use Python with the `pandas` library and the `yfinance` package to fetch historical stock price data from Yahoo

ChatResult(chat_history=[{'content': 'On which days this year was Microsoft Stock higher than $400? Put results in a table.', 'role': 'assistant'}, {'content': "Microsoft's stock closed above $400 on the following dates in 2024: January 24th ($402.56), January 25th ($404.87), January 26th ($403.93), January 29th ($409.72), January 30th ($408.59), February 1st ($403.78), February 2nd ($411.22), February 5th ($405.65), February 6th ($405.49), and February 7th ($414.05).", 'role': 'user'}, {'content': '', 'role': 'assistant'}], summary='', cost=({'total_cost': 0}, {'total_cost': 0}), human_input=[])

### Example 2: Solve multiple tasks with SoM

In [4]:
financial_assistant_2 = autogen.AssistantAgent(
    name="Financial_assistant_2",
    llm_config={"config_list": config_list},
)
writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

financial_assistant_2.register_nested_chats(
    [autogen.Agent, None], [{"recipient": manager, "summary_method": "reflection_with_llm"}]
)
user.initiate_chats(
    [
        {
            "recipient": financial_assistant_1,
            "message": financial_tasks[0],
            "clear_history": True,
            "silent": False,
            "takeaway_method": "reflection_with_llm",
        },
        {
            "recipient": financial_assistant_2,
            "message": financial_tasks[1],
            "takeaway_method": "reflection_with_llm",
        },
        {
            "recipient": writer,
            "message": writing_tasks[0],
        },
    ]
)


********************************************************************************
Start a new chat with the following message: 
On which days this year was Microsoft Stock higher than $400? Put results in a table.

With the following carryover: 


********************************************************************************
User (to Financial_assistant_1):

On which days this year was Microsoft Stock higher than $400? Put results in a table.

--------------------------------------------------------------------------------

********************************************************************************
Start a new chat with the following message: 
On which days this year was Microsoft Stock higher than $400? Put results in a table.

With the following carryover: 


********************************************************************************
Financial_assistant_1 (to chat_manager):

On which days this year was Microsoft Stock higher than $400? Put results in a table.

-----------

{<autogen.agentchat.assistant_agent.AssistantAgent at 0x14ff22b90>: ChatResult(chat_history=[{'content': 'On which days this year was Microsoft Stock higher than $400? Put results in a table.', 'role': 'assistant'}, {'content': "Microsoft's stock closed above $400 on the following dates in 2024: January 24th ($402.56), January 25th ($404.87), January 26th ($403.93), January 29th ($409.72), January 30th ($408.59), February 1st ($403.78), February 2nd ($411.22), February 5th ($405.65), February 6th ($405.49), and February 7th ($414.05).", 'role': 'user'}, {'content': '', 'role': 'assistant'}], summary='', cost=({'total_cost': 0}, {'total_cost': 0}), human_input=[]),
 <autogen.agentchat.assistant_agent.AssistantAgent at 0x14ffdfc10>: ChatResult(chat_history=[{'content': 'Investigate the possible reasons of the stock performance.\nContext: \n', 'role': 'assistant'}, {'content': 'There has been a miscommunication regarding the term "Inner-code-interpreter," which does not match any known st